# Studying a specific cyclone
In this example, we want to study hurricane Wilma (the deepest Atlantic hurricane on record).

In [ ]:
import huracanpy
import matplotlib.pyplot as plt

## Load IBTrACS and subset the specific hurricane
Two subsets of IBTrACS are embedded within HuracanPy: WMO and JTWC.
You can also retrieve the full and last IBTrACS file from the online website.
Default behavior is loading the embedded WMO subset. For more information, see [huracanpy.load guide](../load.ipynb).

In [ ]:
# Here we load the WMO subset. This raises a warning that reminds you of the main
# caveats.
ib = huracanpy.load(source="ibtracs")
# The tracks are loaded as an xarray.Dataset, with one dimension "record" corresponding
# to each point.
# Variables indicate position in space and time, as well as additional attributes such
# as maximum wind speed and minimum slp.
ib

In [ ]:
# Wilma corresponds to index 2005289N18282, so we subset this storm. There are two ways
# of doing this:
# 1. Use warray's where
Wilma = ib.where(ib.track_id == "2005289N18282", drop=True)
# 2. Use huracanpy's sel_id method (more efficient and shorter, but does the same thing)
# Note: the `.hrcn` is called an accessor, and allows you to call HuracanPy functions as
# methods on the xarray objects.
Wilma = ib.hrcn.sel_id("2005289N18282")
# The Wilma object contains only the data for Wilma:
Wilma

## Add category info
You can add the Saffir-Simpson and/or the pressure category of Wilma to the tracks (for full list of available info, see [huracanpy.info](../../api/info.rst)).

In [ ]:
# Add Saffir-Simpson Category
Wilma = Wilma.hrcn.add_saffir_simpson_category(wind_name="wind", wind_units="knots")
# This is stored in the `saffir_simpson_category` variable
Wilma.saffir_simpson_category

In [ ]:
# Add pressure category
Wilma = Wilma.hrcn.add_pressure_category(slp_name="slp")
Wilma.pressure_category

In [ ]:
# Note: Most of the accessor methods have a get_* and an add_* version.
# get_ return the values of what you ask for as a DataArray, while add_ adds it directly
# to the dataset with a default name.
# In the previous case, we could have called get_pressure_category
Wilma.hrcn.get_pressure_category(slp_name="slp")
# we could then save it as a variable, and potentially add it to the dataset separately

## Plot the track and its evolution

In [ ]:
# Plot the track on a map, colored by Saffir-Simpson category
Wilma.hrcn.plot_tracks(
    intensity_var_name="saffir_simpson_category", scatter_kws={"palette": "turbo"}
)

In [ ]:
# Plot intensity time series using matplotlib
fig, axs = plt.subplots(2, sharex=True)
axs[0].plot(Wilma.time, Wilma.wind)
axs[1].plot(Wilma.time, Wilma.slp)
axs[0].set_ylabel("Wind / kn")
axs[1].set_ylabel("SLP / hPa")

## Calculate properties 

### Duration

In [ ]:
Wilma.hrcn.get_track_duration()  # Note duration is in h

### ACE

In [ ]:
## Compute ACE for each point
Wilma = Wilma.hrcn.add_ace(wind_units="knots")
Wilma.ace

In [ ]:
## Plot cumulated ACE
plt.plot(Wilma.time, Wilma.ace.cumsum())

### Translation speed

In [ ]:
# Compute translation speed
Wilma = Wilma.hrcn.add_translation_speed()
# Plot translation speed against latitude
plt.plot(Wilma.lat, Wilma.translation_speed)
plt.xlabel("Latitude / °")
plt.ylabel("Translation speed / m/s")

### Intensification rate

In [ ]:
# Add intensification rate in wind and pressure
Wilma = Wilma.hrcn.add_rate(var_name="wind")
Wilma = Wilma.hrcn.add_rate(var_name="slp")
# NB: The rates will be in unit/s, where unit is the unit of the variable.

In [ ]:
# Plot intensity time series
fig, axs = plt.subplots(2, sharex=True)
axs[0].plot(Wilma.time, Wilma.rate_wind * 3600)  # Convert to kn/h
axs[1].plot(Wilma.time, Wilma.rate_slp * 3600)  # Convert to hPa/h
axs[0].set_ylabel("kn/h")
axs[1].set_ylabel("hPa/h")